In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import joblib
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
O_E = OrdinalEncoder()
train[['id','Target']] = O_E.fit_transform(train[['id','Target']])
train.head(2)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,2.0
1,1.0,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,0.0


In [ ]:
duplicates = train[train.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [id, Marital status, Application mode, Application order, Course, Daytime/evening attendance, Previous qualification, Previous qualification (grade), Nacionality, Mother's qualification, Father's qualification, Mother's occupation, Father's occupation, Admission grade, Displaced, Educational special needs, Debtor, Tuition fees up to date, Gender, Scholarship holder, Age at enrollment, International, Curricular units 1st sem (credited), Curricular units 1st sem (enrolled), Curricular units 1st sem (evaluations), Curricular units 1st sem (approved), Curricular units 1st sem (grade), Curricular units 1st sem (without evaluations), Curricular units 2nd sem (credited), Curricular units 2nd sem (enrolled), Curricular units 2nd sem (evaluations), Curricular units 2nd sem (approved), Curricular units 2nd sem (grade), Curricular units 2nd sem (without evaluations), Unemployment rate, Inflation rate, GDP, Target]
Index: []

[0 rows x 38 columns]


In [ ]:
train.corr()['Target'].sort_values(ascending=False)

,Target
Target,1.000000
Curricular units 2nd sem (approved),0.781452
Curricular units 1st sem (approved),0.725490
Curricular units 2nd sem (grade),0.719036
Curricular units 1st sem (grade),0.661355
Tuition fees up to date,0.415691
Scholarship holder,0.394124
Curricular units 2nd sem (enrolled),0.289165
Curricular units 1st sem (enrolled),0.263657
Curricular units 2nd sem (evaluations),0.214951


In [ ]:
train = train.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)",
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification"
                            ,"Father's occupation", "Inflation rate"])

In [ ]:
test = test.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)",
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification"
                            ,"Father's occupation", "Inflation rate"])

In [ ]:
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
outliers = ((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR)))
print(outliers)

          id  Marital status  Application mode  Application order  Course  \
0      False           False             False              False   False   
1      False           False             False              False   False   
2      False           False             False              False   False   
3      False           False             False              False   False   
4      False           False             False              False   False   
...      ...             ...               ...                ...     ...   
76513  False           False             False              False   False   
76514  False           False             False               True   False   
76515  False            True             False              False   False   
76516  False           False             False              False   False   
76517  False           False             False              False   False   

       Daytime/evening attendance  Previous qualification (grade)  \
0     

In [ ]:
train = train[~((train < (Q1 - 1.5 * IQR)) | (train > (Q3 + 1.5 * IQR))).any(axis=1)]
print(train)

            id  Marital status  Application mode  Application order  Course  \
4          4.0               1                 1                  2    9500   
6          6.0               1                44                  1    9085   
7          7.0               1                 1                  1    9773   
12        12.0               1                 1                  1    9853   
17        17.0               1                17                  1    9238   
...        ...             ...               ...                ...     ...   
76508  76508.0               1                 1                  1    9238   
76509  76509.0               1                 1                  1    9500   
76511  76511.0               1                17                  1    9773   
76512  76512.0               1                 1                  1    9238   
76517  76517.0               1                 1                  1    9773   

       Daytime/evening attendance  Previous qualifi

In [ ]:
print(train.describe())

                 id  Marital status  Application mode  Application order  \
count  20161.000000         20161.0      20161.000000       20161.000000   
mean   38376.805317             1.0         10.890333           1.328605   
std    22071.568909             0.0         14.054021           0.626862   
min        4.000000             1.0          1.000000           1.000000   
25%    19427.000000             1.0          1.000000           1.000000   
50%    38395.000000             1.0          1.000000           1.000000   
75%    57490.000000             1.0         17.000000           1.000000   
max    76517.000000             1.0         53.000000           3.000000   

             Course  Daytime/evening attendance  \
count  20161.000000                     20161.0   
mean    9370.997867                         1.0   
std      292.476048                         0.0   
min     9003.000000                         1.0   
25%     9147.000000                         1.0   
50%     9

In [ ]:
model = CatBoostClassifier(random_state=42)

In [ ]:
features= train.drop(['id', 'Target'],axis=1)
target=train['Target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)

Learning rate set to 0.114151
0:	learn: 1.0106606	test: 1.0109192	best: 1.0109192 (0)	total: 64.5ms	remaining: 1m 4s
1:	learn: 0.9448214	test: 0.9452317	best: 0.9452317 (1)	total: 75.4ms	remaining: 37.6s
2:	learn: 0.8900156	test: 0.8913900	best: 0.8913900 (2)	total: 86.7ms	remaining: 28.8s
3:	learn: 0.8494212	test: 0.8508794	best: 0.8508794 (3)	total: 98.2ms	remaining: 24.5s
4:	learn: 0.8176037	test: 0.8192406	best: 0.8192406 (4)	total: 110ms	remaining: 21.8s
5:	learn: 0.7873225	test: 0.7902823	best: 0.7902823 (5)	total: 121ms	remaining: 20s
6:	learn: 0.7639128	test: 0.7675020	best: 0.7675020 (6)	total: 133ms	remaining: 18.8s
7:	learn: 0.7415764	test: 0.7461954	best: 0.7461954 (7)	total: 144ms	remaining: 17.9s
8:	learn: 0.7246930	test: 0.7295805	best: 0.7295805 (8)	total: 155ms	remaining: 17.1s
9:	learn: 0.7107492	test: 0.7163196	best: 0.7163196 (9)	total: 166ms	remaining: 16.4s
10:	learn: 0.6996872	test: 0.7058718	best: 0.7058718 (10)	total: 178ms	remaining: 16s
11:	learn: 0.6884633	t

In [ ]:
testset = test.drop('id', axis=1)
pred = model.predict(testset)

In [ ]:
predf = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, predf)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 75.35%


In [ ]:
pred.shape =(51012,)

In [ ]:
predset = pd.DataFrame({'id': test['id'], 'Target': pred})
predset.head(3)

,id,Target
0,76518,0.0
1,76519,2.0
2,76520,2.0


In [ ]:
def targetNames(col):
    if col == 0:
        return 'Dropout'
    if col == 1:
        return 'Enrolled'
    if col == 2:
        return 'Graduate'

prediction_df = pd.Series(pred).apply(targetNames)
preddf = pd.DataFrame({'id': test['id'], 'Target': prediction_df})
preddf.head(5)

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Enrolled


In [ ]:
from google.colab import files

# Save the DataFrame to a CSV file
preddf.to_csv('sample_submission.csv', index=False)

In [ ]:
# Download the file
files.download('sample_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>